# Анализ лояльности пользователей Яндекс Афиши

Автор: Павлова Арина

Дата: 16.02.26

### Цели и задачи проекта

Цель: провести исследовательский анализ данных, чтобы понять, какие пользователи с большей вероятностью возвращаются на платформу и делают заказы повторно.

Задачи: 
1. Загрузить данные и познакомиться с ними
2. Провести предобработку данных
3. Провести исследовательский анализ
4. Сформулировать выводы по проделанной работе

### Содержание
1. Загрузка данных и их предобработка
2. Предобработка данных
3. Создание профиля пользователя
4. Исследовательский анализ

4.1. Исследование признаков первого заказа и их связи возвращения на платформу

4.2. Исследование поведения пользователей через показатели выручки и его состава

4.3. Исследование временных характеристик первого заказа и их влияния на повторные покупки

4.4. Корреляционный анализ количества покупок и признаков пользователя

5. Общий вывод и рекомендации

## Этапы выполнения проекта

### 1. Загрузка данных и их предобработка

---

**Задача 1.1:** Напишите SQL-запрос, выгружающий в датафрейм pandas необходимые данные. Используйте следующие параметры для подключения к базе данных `data-analyst-afisha`:

- **Хост** — `rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net`
- **База данных** — `data-analyst-afisha`
- **Порт** — `6432`
- **Аутентификация** — `Database Native`
- **Пользователь** — `praktikum_student`
- **Пароль** — `Sdf4$2;d-d30pp`

Для выгрузки используйте запрос из предыдущего урока и библиотеку SQLAlchemy.

Выгрузка из базы данных SQL должна позволить собрать следующие данные:

- `user_id` — уникальный идентификатор пользователя, совершившего заказ;
- `device_type_canonical` — тип устройства, с которого был оформлен заказ (`mobile` — мобильные устройства, `desktop` — стационарные);
- `order_id` — уникальный идентификатор заказа;
- `order_dt` — дата создания заказа (используйте данные `created_dt_msk`);
- `order_ts` — дата и время создания заказа (используйте данные `created_ts_msk`);
- `currency_code` — валюта оплаты;
- `revenue` — выручка от заказа;
- `tickets_count` — количество купленных билетов;
- `days_since_prev` — количество дней от предыдущей покупки пользователя, для пользователей с одной покупкой — значение пропущено;
- `event_id` — уникальный идентификатор мероприятия;
- `service_name` — название билетного оператора;
- `event_type_main` — основной тип мероприятия (театральная постановка, концерт и так далее);
- `region_name` — название региона, в котором прошло мероприятие;
- `city_name` — название города, в котором прошло мероприятие.

---


In [1]:
# Используйте ячейки типа Code для вашего кода,
# а ячейки типа Markdown для комментариев и выводов

In [2]:
# При необходимости добавляйте новые ячейки для кода или текста

Устанавливаем необходимые библиотеки:

In [3]:
!pip install sqlalchemy 
!pip install psycopg2-binary 
# Необходимо установить данные библиотеки, если они не стоят.
!pip install phik  
!pip install python-dotenv

Импортируем необходимые библиотеки к функции create_engine() из библиотеки PQLAlchemy для подключения к базе данных и выполнения запроса SQL.

In [4]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
import phik
from sqlalchemy import create_engine
from dotenv import load_dotenv

Данные для подключения к базе данных data-analyst-afisha: в объект db_config

In [5]:
db_config = {'user': 'praktikum_student', # имя пользователя
             'pwd': 'Sdf4$2;d-d30pp', # пароль
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432, # порт подключения
             'db': 'data-analyst-afisha' # название базы данных
             } 

Строка подключения, которая содержит информацию об используемой СУБД, а также данные авторизации

In [6]:
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(
    db_config['user'],
    db_config['pwd'],
    db_config['host'],
    db_config['port'],
    db_config['db'],
)

In [7]:
#Создаем соединение

engine = create_engine(connection_string) 

In [8]:
query = """-- Настройка параметра synchronize_seqscans важна для проверки
WITH set_config_precode AS (
  SELECT set_config('synchronize_seqscans', 'off', true)
)

-- Напишите ваш запрос ниже
SELECT p.user_id, p.device_type_canonical, p.order_id, p.created_dt_msk AS order_dt, p.created_ts_msk AS order_ts, p.currency_code, p.revenue, p.tickets_count,
p.created_dt_msk::date - LAG(p.created_dt_msk::date) OVER (PARTITION BY p.user_id ORDER BY p.created_dt_msk) AS days_since_prev,
e.event_id,
e.event_name_code AS event_name,
e.event_type_main,
p.service_name,
r.region_name,
c.city_name
FROM afisha.purchases p 
INNER JOIN afisha.events e ON p.event_id = e.event_id
INNER JOIN afisha.city c ON e.city_id = c.city_id
INNER JOIN afisha.regions r ON c.region_id = r.region_id
WHERE p.device_type_canonical IN ('mobile', 'desktop') AND e.event_type_main != 'фильм'
ORDER BY p.user_id
"""

In [9]:
# Запишем получившийся результат SQL-запроса в датафрейм.
df = pd.read_sql_query(query, con=engine)

---

**Задача 1.2:** Изучите общую информацию о выгруженных данных. Оцените корректность выгрузки и объём полученных данных.

Предположите, какие шаги необходимо сделать на стадии предобработки данных — например, скорректировать типы данных.

Зафиксируйте основную информацию о данных в кратком промежуточном выводе.

---

In [10]:
# Познакомимся с данными датасета
df.head()

,user_id,device_type_canonical,order_id,order_dt,order_ts,currency_code,revenue,tickets_count,days_since_prev,event_id,event_name,event_type_main,service_name,region_name,city_name
0,0002849b70a3ce2,mobile,4359165,2024-08-20,2024-08-20 16:08:03,rub,1521.94,4,NaN,169230,f0f7b271-04eb-4af6-bcb8-8f05cf46d6ad,театр,Край билетов,Каменевский регион,Глиногорск
1,0005ca5e93f2cf4,mobile,7965605,2024-07-23,2024-07-23 18:36:24,rub,289.45,2,NaN,237325,40efeb04-81b7-4135-b41f-708ff00cc64c,выставки,Мой билет,Каменевский регион,Глиногорск
2,0005ca5e93f2cf4,mobile,7292370,2024-10-06,2024-10-06 13:56:02,rub,1258.57,4,75.0,578454,01f3fb7b-ed07-4f94-b1d3-9a2e1ee5a8ca,другое,За билетом!,Каменевский регион,Глиногорск
3,000898990054619,mobile,1139875,2024-07-13,2024-07-13 19:40:48,rub,8.49,2,NaN,387271,2f638715-8844-466c-b43f-378a627c419f,другое,Лови билет!,Североярская область,Озёрск
4,000898990054619,mobile,972400,2024-10-04,2024-10-04 22:33:15,rub,1390.41,3,83.0,509453,10d805d3-9809-4d8a-834e-225b7d03f95d,стендап,Билеты без проблем,Озернинский край,Родниковецк


In [11]:
# Выводим информацию о датафрейме
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 290611 entries, 0 to 290610
Data columns (total 15 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   user_id                290611 non-null  object        
 1   device_type_canonical  290611 non-null  object        
 2   order_id               290611 non-null  int64         
 3   order_dt               290611 non-null  datetime64[ns]
 4   order_ts               290611 non-null  datetime64[ns]
 5   currency_code          290611 non-null  object        
 6   revenue                290611 non-null  float64       
 7   tickets_count          290611 non-null  int64         
 8   days_since_prev        268678 non-null  float64       
 9   event_id               290611 non-null  int64         
 10  event_name             290611 non-null  object        
 11  event_type_main        290611 non-null  object        
 12  service_name           290611 non-null  obje

In [12]:
# Выводим размер датафрейма 
df.shape

(290611, 15)

In [13]:
# создаем копию датасета до преобразования для возможности проверить сделанные изменения после предобработки

temp = df.copy() 
len(temp)

290611

###### ПРОМЕЖУТОЧНЫЙ ВЫВОД:
Данные представляют собой 15 столбцов и 290611 строк, которые хранят информацию данных о пользователях, их заказах.

Название столбоцов приведены к стилю snake_case. В столбце revenue данный стиль не используется.

order_ts,order_id- имеют тип данных datetime64
revenue,days_since_prev тип данных float64
event_id,tickets_count,order_id тип данных int64, проведем оптимизацию данных.
user_id,device_type_canonical,currency_code ,event_name,event_type_main,service_name,region_name,city_name -тип данных object

days_since_prev - имеет пропуски.
Однако следует проверить и другие столбцы: в них могут встречаться значения-индикаторы, которые будут говорить об отсутствии данных.

Первичное знакомство показывает, что данные хорошего качества — в них практически нет пропущенных значений, а сами данные соответствуют описанию и выглядят корректными.

---

###  2. Предобработка данных

Выполните все стандартные действия по предобработке данных:

---

**Задача 2.1:** Данные о выручке сервиса представлены в российских рублях и казахстанских тенге. Приведите выручку к единой валюте — российскому рублю.

Для этого используйте датасет с информацией о курсе казахстанского тенге по отношению к российскому рублю за 2024 год — `final_tickets_tenge_df.csv`. Его можно загрузить по пути `https://code.s3.yandex.net/datasets/final_tickets_tenge_df.csv')`

Значения в рублях представлено для 100 тенге.

Результаты преобразования сохраните в новый столбец `revenue_rub`.

---


In [14]:
#Загружаем данные о курсе тенге
rates_df = pd.read_csv('https://code.s3.yandex.net/datasets/final_tickets_tenge_df.csv')

Объединим таблицы:

In [15]:
# Объединение сделаем  по "order_id" из основной таблицы и  "data" их таблицы курсов
df = df.merge(rates_df[['data', 'curs']], left_on='order_dt', right_on='data', how='left')

ValueError: You are trying to merge on datetime64[ns] and object columns. If you wish to proceed you should use pd.concat

In [ ]:
#Удаляем лишнюю колонку 'data', которая дублирует 'order_dt'
df = df.drop(columns=['data'])

In [ ]:
#Рассчитываем выручку в рублях
df['revenue_rub'] = df['revenue'].where(
    df['currency_code'] == 'rub', 
    (df['revenue'] / 100) * df['curs']
)

In [ ]:
#Удалим столбец курс,после приведения к единной валюте 
df = df.drop(columns=['curs'])

In [ ]:
#Проверим результат
print("Колонки в df:", df.columns)
display(df.head())

---

**Задача 2.2:**

- Проверьте данные на пропущенные значения. Если выгрузка из SQL была успешной, то пропуски должны быть только в столбце `days_since_prev`.
- Преобразуйте типы данных в некоторых столбцах, если это необходимо. Обратите внимание на данные с датой и временем, а также на числовые данные, размерность которых можно сократить.
- Изучите значения в ключевых столбцах. Обработайте ошибки, если обнаружите их.
    - Проверьте, какие категории указаны в столбцах с номинальными данными. Есть ли среди категорий такие, что обозначают пропуски в данных или отсутствие информации? Проведите нормализацию данных, если это необходимо.
    - Проверьте распределение численных данных и наличие в них выбросов. Для этого используйте статистические показатели, гистограммы распределения значений или диаграммы размаха.
        
        Важные показатели в рамках поставленной задачи — это выручка с заказа (`revenue_rub`) и количество билетов в заказе (`tickets_count`), поэтому в первую очередь проверьте данные в этих столбцах.
        
        Если обнаружите выбросы в поле `revenue_rub`, то отфильтруйте значения по 99 перцентилю.

После предобработки проверьте, были ли отфильтрованы данные. Если были, то оцените, в каком объёме. Сформулируйте промежуточный вывод, зафиксировав основные действия и описания новых столбцов.

---

In [ ]:
# Выводим названия столбцов датафрейма
df.columns

In [ ]:
missing_counts = df.isna().sum()
missing_percent = df.isna().mean() * 100

pd.DataFrame({
    "missing_count": missing_counts,
    "missing_percent": missing_percent
}).sort_values(by="missing_percent", ascending=False)

In [ ]:
def show_missing_stats(tmp0):
    """
    Функция для отображения статистики пропущенных значений в DataFrame.
    """
    missing_stats = pd.DataFrame({
        'Кол-во пропусков': tmp0.isnull().sum(),
        'Доля пропусков': tmp0.isnull().mean()*100
    })
    missing_stats = missing_stats[missing_stats['Кол-во пропусков'] > 0]
    
    if missing_stats.empty:
        return "Пропусков в данных нет"
    
    # Форматируем при выводе через Styler
    return (missing_stats.style.format({'Доля пропусков': '{:.4f}'}).background_gradient(cmap='coolwarm'))
show_missing_stats(df)

In [ ]:
#Выведем размер датасета до обработки
original_rows = len(df)
print(f"Исходный размер датасета: {original_rows} строк")

In [ ]:
# Cоздаим копию датасета до преобразования для возможности проверить сделанные изменения после предобработки
tmp = df.copy() 
len(tmp)

In [ ]:
# Анализируем тип данных
print("\nТипы данных столбцов:")
print(
    df.dtypes
        .sort_values()         
)
print('\n')

In [ ]:
#Оптимизируем тип данных
for column in ['revenue','days_since_prev','revenue_rub']:
    df[column] = pd.to_numeric(df[column],errors='coerce',
                                    downcast='float')

In [ ]:
#Оптимизируем целочисленный тип данных
for column in ['order_id','tickets_count','event_id'
               ]:
    df[column] = pd.to_numeric(df[column],
                                    downcast='integer')

После внесения изменений проверяем тип данных:

In [ ]:
print("\nТипы данных столбцов после изменений:")
print(
    df.dtypes
        .sort_values()         
)
print('\n')

 Промежуточный вывод: 
Для экономии памяти понизили разрядность столбцов float64 до float32 и int64 до int8, оптимизация на дальнейшее исследование не влияет. Столбцы с типом данных object оставили без изменний.

In [ ]:
# Проверяем уникальные значения в столбцах
for column in ['tickets_count', 'days_since_prev', 'event_id', 'revenue']:
    print(f'Уникальные значения в столбце {column}:')
    print(df[column].sort_values().unique())
    print()

Проверим уникальные значения в категориальных столбцах до их преобразования

In [ ]:
df['device_type_canonical'].unique()

In [ ]:
df['currency_code'].unique()

In [ ]:
df['event_name'].unique()

In [ ]:
df['event_type_main'].unique()

In [ ]:
df['service_name'].unique()

In [ ]:
df['region_name'].unique()

In [ ]:
df['city_name'].unique()

In [ ]:
df['order_dt'].unique()

In [ ]:
# Проверим полные дубликаты (строки, которые полностью идентичны)
duplicates_count = df.duplicated().sum()
print(f"Найдено полных дубликатов: {duplicates_count}")

# Если они есть — удалим их
if duplicates_count > 0:
    df = df.drop_duplicates().reset_index(drop=True)
    print("Полные дубликаты удалены.")

# Проверка «неполных» дубликатов (например, один и тот же пользователь сделал заказ в одну секунду)
# Посмотрим, есть ли дубли по комбинации 'user_id' и 'order_ts'
partial_duplicates = df.duplicated(subset=['user_id', 'order_ts']).sum()
print(f"Найдено подозрительных дубликатов по (user + время): {partial_duplicates}")

In [ ]:
# Удаляим дубликаты по пользователю и времени.
df = df.drop_duplicates(subset=['user_id', 'order_ts']).reset_index(drop=True)
print(f"Дубликаты удалены. Новый размер таблицы: {df.shape}")

In [ ]:
 # Cоздаем список столбцов 
subset = ['user_id', 'order_ts', 'revenue', 'tickets_count', 'service_name']

# Ищем дубликаты
duplicates = df[df.duplicated(subset=subset, keep=False)]

# Выводим результат
if not duplicates.empty:
    print(f"Найдено строк с дублями: {len(duplicates)}")
    # Сортируем, чтобы одинаковые строки стояли рядом
    display(duplicates.sort_values(by=['user_id', 'order_ts']))
    
    # Считаем количество лишних копий
    extra_rows = df.duplicated(subset=subset).sum()
    print(f"Количество лишних строк (копий): {extra_rows}")
else:
    print("Дубликатов по выбранным столбцам не обнаружено.")

In [ ]:
#Проверим наличие явных дубликатов
df.duplicated().sum()

In [ ]:
#Проверим наличие неявных дубликатов по совпадению в столбцах
duplicates=df.duplicated(subset=['device_type_canonical', 'event_type_main', 'service_name', 'city_name']).sum()

In [ ]:
# Предварительно нормализуем данные
# Приводим к нижнему регистру
for column in ['device_type_canonical', 'event_type_main', 'service_name', 'city_name']:
    df[column]=df[column].str.lower().str.strip()
       
df.head()

# Проверим уникальные значения в одном из ключевых столбцов
print("Уникальные типы устройств:", df['device_type_canonical'].unique())

In [ ]:
#Проверим наличие неявных дубликатов по совпадению в столбцах 
duplicates=df.duplicated(subset=['device_type_canonical', 'event_type_main', 'service_name', 'city_name']).sum()

In [ ]:
#Удалим дубликаты
df = df.drop_duplicates(subset=subset, keep='first').reset_index(drop=True)

In [ ]:
#Выведем размер датасета после обработки
original_rows = len(df)
print(f"Размер датасета после обработки: {original_rows} строк")

In [ ]:
# Проверим сколько удалено строк датасета
a, b = len(tmp), len(df)
print(" Было строк в исходном датасете", a,
      '\n', "Осталось строк в датасете после обработки", b,
      '\n', "Удалено строк в датасете после обработки", a-b,
      '\n', "Процент потерь", round((a-b)/a*100, 2))

In [ ]:
df.describe()

Согласно статистическим данным минимальное значение имеет отрицательное значение по выручке.

In [ ]:
# Абсолютное значение записей с отрицательной выручкой в рублях
negative_revenue_count = df[df['revenue_rub'] < 0]['revenue_rub'].count()
print(f"Количество строк с отрицательной выручкой: {negative_revenue_count}")

In [ ]:
#Относительное значение записей с отрицательной выручкой в рублях
print(f'Относительное распределение данных ')
print((df['revenue_rub']<0).value_counts(normalize=True)*100)

In [ ]:
# Строим диаграмму размаха методом boxplot()
boxplot = df.boxplot(column='revenue_rub',
                     vert=False, 
                     showfliers=False,
                     figsize=(12, 10))

# Добавляем заголовок и метку оси X
boxplot.set_title('Распределение выручки в рублях')
boxplot.set_xlabel('Выручка')

plt.show()

In [ ]:
# Строим диаграмму размаха методом boxplot()
boxplot = df.boxplot(column='revenue_rub',
                     vert=False, 
                     figsize=(12, 10))

# Добавляем заголовок и метку оси X
boxplot.set_title('Распределение выручки в рублях')
boxplot.set_xlabel('Выручка')

plt.show()

По диаграмме размаха наблюдаются выбросы в отрицательную сторону. В данных есть отрицательные значения выручки, возможные ошибки или возвраты, которые исказят сумму общих продаж, поэтому их необходимо удалить.

In [ ]:
#Удаленим аномалий (Отрицательная выручка)
tmo=len(df)
df = df[df['revenue_rub'] >0]

print(f"Удалено строк с некорректной выручкой: {tmo - len(df)}")

Анализ распределения и фильтрация выбросов по 99 перцентилю. Сверхвысокие значения (выбросы) в revenue_rub могут сильно завышать средний чек. Мы отсекаем 1% самых экстремальных значений, чтобы видеть реальную картину поведения большинства пользователей.

In [ ]:
# Считаем 99-й перцентиль для выручки
revenue_limit = np.percentile(df['revenue_rub'], 99)

In [ ]:
# Фильтруем данные: оставляем только те, что не превышают лимит
tmr = len(df)
df = df[df['revenue_rub'] <= revenue_limit]

print(f"Порог 99-го перцентиля для выручки: {revenue_limit:.2f} руб.")
print(f"Удалено строк-выбросов: {tmr - len(df)}")

In [ ]:
#Посмотрим на статистику после очистки
df[['revenue_rub','tickets_count']].describe()

In [ ]:
# Строим диаграмму размаха методом boxplot()
boxplot = df.boxplot(column='revenue_rub',
                     vert=False, 
                     figsize=(12, 10))

# Добавляем заголовок и метку оси X
boxplot.set_title('Распределение выручки в рублях')
boxplot.set_xlabel('Выручка')

plt.show()

In [ ]:
# Итоговая проверка
tms = len(df)
total_deleted = len(temp) - tms
percent_deleted = (total_deleted / len(temp)) * 100

print(f"Общее количество удаленных строк: {total_deleted}")
print(f"Процент потерь: {percent_deleted:.2f}%")

**Промежуточный вывод:** После проведения предобработки данных были выявлено:

Пропуски в столбце days_since_prev в количестве 21,933 пропуска (7.5% данных) оставили как есть;

Понизили разрядность у столбцов revenue,days_since_prev,revenue_rub до float 32 для экономии места;

Понизили разрядность у целых чисел order_id, tickets_count,event_id понизили до int32 и int8.

Удалили аномальные данные (отрицательную выручку), дубликаты за двоеенные по времени и пользователю.

Отфильтровали данные по 99 проценталю, исключив аномальные выбросы по выручки.

Общее количество потерь составило 4% от общего объема , что находится в допустимом диапозоне.

---

### 3. Создание профиля пользователя

В будущем отдел маркетинга планирует создать модель для прогнозирования возврата пользователей. Поэтому сейчас они просят вас построить агрегированные признаки, описывающие поведение и профиль каждого пользователя.

---

**Задача 3.1.** Постройте профиль пользователя — для каждого пользователя найдите:

- дату первого и последнего заказа;
- устройство, с которого был сделан первый заказ;
- регион, в котором был сделан первый заказ;
- билетного партнёра, к которому обращались при первом заказе;
- жанр первого посещённого мероприятия (используйте поле `event_type_main`);
- общее количество заказов;
- средняя выручка с одного заказа в рублях;
- среднее количество билетов в заказе;
- среднее время между заказами.

После этого добавьте два бинарных признака:

- `is_two` — совершил ли пользователь 2 и более заказа;
- `is_five` — совершил ли пользователь 5 и более заказов.

**Рекомендация:** перед тем как строить профиль, отсортируйте данные по времени совершения заказа.

---


In [ ]:
# Отсортируем данные по времени совершения заказа 
df.sort_values('order_dt')

Построим профиль пользователя:

In [ ]:
# Сортируем для корректной работы first/last
df = df.sort_values('order_ts')

# Группируем данные по пользователю
user_profile = (df.groupby('user_id')
                .agg(
                    first_order_date=('order_dt', 'first'),           # Дата первого заказа
                    last_order_date=('order_dt', 'last'),             # Дата последнего заказа
                    first_device=('device_type_canonical', 'first'),  # Устройство первого заказа
                    first_region=('region_name', 'first'),            # Регион первого заказа
                    first_partner=('service_name', 'first'),          # Билетный партнер первого заказа
                    first_genre=('event_type_main', 'first'),         # Жанр первого мероприятия
                    total_orders=('order_id', 'count'),               # Общее количество заказов       
                    avg_revenue=('revenue_rub', 'mean'),              # Средняя выручка (в рублях)
                    avg_tickets=('tickets_count', 'mean'),            # Среднее кол-во билетов
                    avg_days_between=('days_since_prev', 'mean')     # Среднее время между заказами
                    )
                .reset_index()
                .assign(
                    avg_days_between=lambda x: x['avg_days_between'].fillna(0),
                    customer_lifetime_days=lambda x: (x['last_order_date'] - x['first_order_date']).dt.days.fillna(0),
                    recency_days=lambda x: (pd.Timestamp.now().normalize() - x['last_order_date']).dt.days
                ))

In [ ]:
 #Добавим бинарные признаки (1 - да, 0 - нет)
# Совершил ли пользователь 2 и более заказа
user_profile['is_two'] = (user_profile['total_orders'] >= 2).astype(int)

# Совершил ли пользователь 5 и более заказов
user_profile['is_five'] = (user_profile['total_orders'] >= 5).astype(int)

In [ ]:
#Выводим результат
print(f"Количество уникальных профилей: {len(user_profile)}")
user_profile.head()

**Промежуточный вывод**: Построили профиль пользователя по ключевым моментам, сгруппировав все заказы одного пользователя.

---

**Задача 3.2.** Прежде чем проводить исследовательский анализ данных и делать выводы, важно понять, с какими данными вы работаете: насколько они репрезентативны и нет ли в них аномалий.

Используя данные о профилях пользователей, рассчитайте:

- общее число пользователей в выборке;
- среднюю выручку с одного заказа;
- долю пользователей, совершивших 2 и более заказа;
- долю пользователей, совершивших 5 и более заказов.

Также изучите статистические показатели:

- по общему числу заказов;
- по среднему числу билетов в заказе;
- по среднему количеству дней между покупками.

По результатам оцените данные: достаточно ли их по объёму, есть ли аномальные значения в данных о количестве заказов и среднем количестве билетов?

Если вы найдёте аномальные значения, опишите их и примите обоснованное решение о том, как с ними поступить:

- Оставить и учитывать их при анализе?
- Отфильтровать данные по какому-то значению, например, по 95-му или 99-му перцентилю?

Если вы проведёте фильтрацию, то вычислите объём отфильтрованных данных и выведите статистические показатели по обновлённому датасету.

In [ ]:
# Рассчитаем общее число пользователей в выборке
total_users=len(user_profile)

# Рассчитаем среднюю выручку одного заказа
avg_revenue_order= user_profile['avg_revenue'].mean()

# Найдем долю пользователей совершивших 2 и более заказа;
share_is_two= user_profile['is_two'].mean()

# Найдем долю пользователей совершивших 5 и более заказов
share_is_five= user_profile['is_five'].mean()

print(f"Общее число пользователей: {total_users}")
print(f"Средняя выручка с одного заказа: {avg_revenue_order:.2f} руб.")
print(f"Доля пользователей с 2+ заказами: {share_is_two:.2%}")
print(f"Доля пользователей с 5+ заказами: {share_is_five:.2%}")

In [ ]:
# Изучаем статистику по ключевым признакам
stats = ['total_orders', 'avg_tickets', 'avg_days_between']
user_profile[stats].describe()

In [ ]:
# Строим столбчатую диаграмму
stats = ['total_orders', 'avg_tickets', 'avg_days_between']

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for i, col in enumerate(stats):
    user_profile[col].hist(bins=50, ax=axes[i])
    axes[i].set_title(f'Распределение {col}')
    axes[i].set_xlabel(col)
    axes[i].set_ylabel('Частота')

plt.tight_layout()
plt.show()

In [ ]:
# Создаем отдельные горизонтальные boxplot для каждой метрики
fig, axes = plt.subplots(3, 1, figsize=(12, 10))

# 1. Количество заказов
user_profile.boxplot(column='total_orders', ax=axes[0], vert=False)
axes[0].set_title('Распределение количества заказов на пользователя')
axes[0].set_xlabel('Количество заказов')

# 2. Среднее количество билетов
user_profile.boxplot(column='avg_tickets', ax=axes[1], vert=False)
axes[1].set_title('Распределение среднего количества билетов в заказе')
axes[1].set_xlabel('Среднее количество билетов')

# 3. Среднее время между заказами
user_profile.boxplot(column='avg_days_between', ax=axes[2], vert=False)
axes[2].set_title('Распределение среднего времени между заказами')
axes[2].set_xlabel('Дни')

plt.tight_layout()
plt.show()

**Промежуточный вывод**: В total_orders (кол-во заказов) или avg_tickets (среднее число билетов) максимальные значения превышают средние, что также показано на графиках. Отфильтруем по 99 переценталю.

In [ ]:
# Определяем пороги для фильтрации по 99 перцентилю
orders_limit = np.percentile(user_profile['total_orders'], 99)
tickets_limit = np.percentile(user_profile['avg_tickets'], 99)

# Фильтруем профили
user_profile_filtered = user_profile[
    (user_profile['total_orders'] <= orders_limit) & 
    (user_profile['avg_tickets'] <= tickets_limit)
]

# Считаем объем отфильтрованных данных
deleted_profiles = len(user_profile) - len(user_profile_filtered)
percent_deleted_profiles = (deleted_profiles / len(user_profile)) * 100

print(f"Порог по количеству заказов (99%): {orders_limit}")
print(f"Порог по среднему числу билетов (99%): {tickets_limit:.2f}")
print(f"Удалено аномальных профилей: {deleted_profiles} ({percent_deleted_profiles:.2f}%)")

# Статистика по обновленному датасету
user_profile_filtered[stats].describe()

**Промежуточный вывод**: После фильтрации у нас осталось 21 700 уникальных профилей, данное количество достаточно для анализа. Согласно данным 62% пользователей совершили 2 и более заказов, и больше 29% пользователей сделали заказы от 5 и больше, что показывает о лояльности клиентов и сформировавшиеся базе клиентов. Средняя выручка одного заказа приблизительно 552 руб., Данные по пользователю содежрат аномальные выбросы, такие как пользователь совершил 9972 заказ за предоставленный период. Средний количество до 13. Провели фильтрацию по 99 перцентали, с помощью,которой убрали пользователей с 120 заказами или более 5 билетов за раз.После фильтрации было удале 411 профиля это 1.89% от общего объема. Среднее количество заказов снизилось с с 13 до 6, что близко к значению обычного пользователя.

---

### 4. Исследовательский анализ данных

Следующий этап — исследование признаков, влияющих на возврат пользователей, то есть на совершение повторного заказа. Для этого используйте профили пользователей.



#### 4.1. Исследование признаков первого заказа и их связи с возвращением на платформу

Исследуйте признаки, описывающие первый заказ пользователя, и выясните, влияют ли они на вероятность возвращения пользователя.

---

**Задача 4.1.1.** Изучите распределение пользователей по признакам.

- Сгруппируйте пользователей:
    - по типу их первого мероприятия;
    - по типу устройства, с которого совершена первая покупка;
    - по региону проведения мероприятия из первого заказа;
    - по билетному оператору, продавшему билеты на первый заказ.
- Подсчитайте общее количество пользователей в каждом сегменте и их долю в разрезе каждого признака. Сегмент — это группа пользователей, объединённых определённым признаком, то есть объединённые принадлежностью к категории. Например, все клиенты, сделавшие первый заказ с мобильного телефона, — это сегмент.
- Ответьте на вопрос: равномерно ли распределены пользователи по сегментам или есть выраженные «точки входа» — сегменты с наибольшим числом пользователей?

---


In [ ]:
# Список признаков для анализа распределения
features = {
    'first_genre': 'Тип мероприятия',
    'first_device': 'Тип устройства',
    'first_region': 'Регион',
    'first_partner': 'Билетный оператор'
}

# Проходим циклом по признакам для подсчета количества и долей
for col, name in features.items():
    print(f"\n--- Распределение по признаку: {name} ---")
    
    # Группируем и считаем количество пользователей
    segment_data = user_profile_filtered.groupby(col).agg(
        user_count=('user_id', 'count')
    ).reset_index()
    
    # Рассчитываем долю каждого сегмента в процентах
    total_segment_users = segment_data['user_count'].sum()
    segment_data['share_percent'] = (segment_data['user_count'] / total_segment_users) * 100
    
    # Сортируем по количеству пользователей для наглядности
    segment_data = segment_data.sort_values(by='user_count', ascending=False)
    
    # Выводим результат
    display(segment_data.style.format({'share_percent': '{:.2f}%'}))

Распределение по типу первого мероприятия (Жанр)

In [ ]:
# Настройка стиля графиков
sns.set_theme(style="whitegrid")

# Группировка
genre_data = user_profile_filtered.groupby('first_genre')['user_id'].count().sort_values(ascending=False).reset_index()

# Построение диаграммы
plt.figure(figsize=(12, 6))
sns.barplot(data=genre_data, x='user_id', y='first_genre', palette='viridis')
plt.title('Распределение пользователей по жанру первого мероприятия', fontsize=15)
plt.xlabel('Количество пользователей')
plt.ylabel('Жанр')
plt.show()

# Вывод таблицы с долями
genre_data['share %'] = (genre_data['user_id'] / genre_data['user_id'].sum() * 100).round(2)
display(genre_data)

Распределение по типу устройства.

In [ ]:
# Группировка
device_data = user_profile_filtered.groupby('first_device')['user_id'].count().sort_values(ascending=False).reset_index()

# Построение диаграммы (круговая диаграмма тут будет нагляднее)
plt.figure(figsize=(8, 8))
plt.pie(device_data['user_id'], labels=device_data['first_device'], autopct='%1.1f%%', colors=['#66b3ff','#99ff99'], startangle=140)
plt.title('Доли устройств при первом заказе', fontsize=15)
plt.show()

display(device_data)

Распределение по регионам (Топ-10)Регионов может быть много, поэтому мы выведем 10 самых крупных.

In [ ]:
# Группировка и выбор Топ-10
region_data = user_profile_filtered.groupby('first_region')['user_id'].count().sort_values(ascending=False).head(10).reset_index()

# Построение диаграммы
plt.figure(figsize=(12, 6))
sns.barplot(data=region_data, x='user_id', y='first_region', palette='magma')
plt.title('Топ-10 регионов по количеству новых пользователей', fontsize=15)
plt.xlabel('Количество пользователей')
plt.ylabel('Регион')
plt.show()

display(region_data)

Распределение по билетному оператору (Топ-10)

In [ ]:
# Группировка и выбор Топ-10
partner_data = user_profile_filtered.groupby('first_partner')['user_id'].count().sort_values(ascending=False).head(10).reset_index()

# Построение диаграммы
plt.figure(figsize=(12, 6))
sns.barplot(data=partner_data, x='user_id', y='first_partner', palette='coolwarm')
plt.title('Топ-10 билетных операторов (первый заказ)', fontsize=15)
plt.xlabel('Количество пользователей')
plt.ylabel('Оператор')
plt.show()

display(partner_data)

**Промежуточный вывод**: Распределение пользователей по сегментам является неравномерным, что позволяет выделить четко выраженные лидерские категории в каждом признаке.

По типу мероприятия большей популярностью пользуются концерты(45%), другое(24%) и театр(20%). Жанры такие как стендап , елки ,выставки и т.д это примерно 11% покупок менее популярны. Сервисом больше пользуются для покупки концерта и театра.
По типу устройств заказы совершают через мобильное устройство 17648 заказов это 82.9% от общего числа заказов через устройство. Все дополнительные решение по продажам билетов и акций на них, пользованием, должны быть на целены на пользователя с мобильного устройства.
По региону мепрориятия первого заказа большего всего делают Каменевский регион это 6919 заказов.За ним следует Североярская область.
Билетные операторы Лидером среди операторов является «билеты без проблем» (4,896 пользователей). Группа из четырех операторов («билеты без проблем», «лови билет!», «мой билет», «билеты в руки») обеспечивает основной приток первичных заказов.Сотрудничество с ключевыми операторами-лидерами является определяющим фактором для привлечения новой аудитории.
Итоговый портрет пользователя проживающий в Каменевском регионе купивший билет через мобильное устройство на концерт через оператора "билеты без проблем"

---

**Задача 4.1.2.** Проанализируйте возвраты пользователей:

- Для каждого сегмента вычислите долю пользователей, совершивших два и более заказа.
- Визуализируйте результат подходящим графиком. Если сегментов слишком много, то поместите на график только 10 сегментов с наибольшим количеством пользователей. Такое возможно с сегментами по региону и по билетному оператору.
- Ответьте на вопросы:
    - Какие сегменты пользователей чаще возвращаются на Яндекс Афишу?
    - Наблюдаются ли успешные «точки входа» — такие сегменты, в которых пользователи чаще совершают повторный заказ, чем в среднем по выборке?

При интерпретации результатов учитывайте размер сегментов: если в сегменте мало пользователей (например, десятки), то доли могут быть нестабильными и недостоверными, то есть показывать широкую вариацию значений.

---


In [ ]:
# Группируем по сегменту и считаем долю вернувшихся
retention_genre = user_profile.groupby('first_genre').agg(
        total_users=('user_id', 'count'),
        retention_rate=('is_two', 'mean')
    ).reset_index()


# Сортируем по доле возврата для графика
retention_genre = retention_genre.sort_values('retention_rate', ascending=False)

retention_genre    

In [ ]:
# Сортируем по доле возврата для графика
retention_genre = retention_genre.sort_values('retention_rate', ascending=False)

In [ ]:
#Строим визуализацию
plt.figure(figsize=(12, 6)) 
sns.barplot(
        data=retention_genre, 
        x='retention_rate', 
        y='first_genre', 
        palette='RdYlGn'
    )
plt.title('Возврат пользователей по жанру первого мероприятия', fontsize=14)
plt.xlabel('Доля вернувшихся (2+ заказа)')
plt.ylabel(None)
plt.axvline(0.6167, color='red', linestyle='--', label='Среднее по выборке (61.67%)')
plt.tight_layout()
plt.legend()
plt.show()


In [ ]:
# Группируем по сегменту и считаем долю вернувшихся по регионам
retention_region = user_profile.groupby('first_region').agg(
        total_users=('user_id', 'count'),
        retention_rate=('is_two', 'mean')
    ).reset_index()
# Сортируем по доле возврата для графика
retention_region = retention_region.sort_values('total_users', ascending=False).head(10)

retention_region.head(10)

In [ ]:
# Сортируем по доле возврата для графика
retention_region = retention_region.sort_values('retention_rate', ascending=False).head(10)

In [ ]:
#Строим визуализацию
plt.figure(figsize=(12, 6)) 
sns.barplot(
        data=retention_region, 
        x='retention_rate', 
        y='first_region', 
        palette='RdYlGn'
    )
plt.title('Возврат пользователей по регионам (Топ-10 размеру)', fontsize=14)
plt.xlabel('Доля вернувшихся (2+ заказа)')
plt.ylabel(None)
plt.axvline(0.6167, color='red', linestyle='--', label='Среднее по выборке (61.67%)')
plt.tight_layout()
plt.legend()
plt.show()

In [ ]:
# Группируем по сегменту и считаем долю вернувшихся
retention_partner = user_profile.groupby('first_partner').agg(
        total_users=('user_id', 'count'),
        retention_rate=('is_two', 'mean')
    ).reset_index()
# Сортируем по доле возврата для графика
retention_partner = retention_partner.sort_values('total_users', ascending=False).head(10)

retention_partner.head(10)

In [ ]:
# Сортируем по доле возврата для графика
retention_partner = retention_partner.sort_values('retention_rate', ascending=False).head(10)

In [ ]:
#Строим визуализацию
plt.figure(figsize=(12, 6)) 
sns.barplot(
        data=retention_partner, 
        x='retention_rate', 
        y='first_partner', 
        palette='RdYlGn'
    )
plt.title('Возврат пользователей по операторам (Топ-10 по размеру)', fontsize=14)
plt.xlabel('Доля вернувшихся (2+ заказа)')
plt.ylabel(None)
plt.axvline(0.6167, color='red', linestyle='--', label='Среднее по выборке (61.67%)')
plt.tight_layout()
plt.legend()
plt.show()

**Промежуточные выводы**: Про анализировав данные по сегментам можно сказать следующее:

Покупатели чаще всего возвращаются за повторной покупкой билетов после первого посещения выставки(65%) , театра(64%) или концерта(62%). Меньше всего возратов за повторной покупкой после елок(57%) и спорта(55%) можно предположить из-за сезоности мероприятий ,либо категории людей не соответсвует интересам данных видов мероприятий меньше.
По регионам после первого мероприятие чаще возвращаются клиенты в Шанырском регионе (70%) и Светополянский округ(66%), Широковская область(65%), менее востребовано Озернинский край(55%) и Малиновоярский округ(56%). Предположительно , что разница в спросе на повторные покупки в связи с разным ассортиментом мероприятий.
В качестве оператора по приобретению билетов чаще выбирают "край билетов"(66%),"дом культуры"(65)%. Менее востребованы "мой билет"(61%),"билеты без проблем"(60%). Однако стоит отметить , что крупный оператор "билеты без проблем" при большой аудитории имеет показатель возратов ниже среднего по выборке. Возможный выбор покупателей другого оператора связано с неудобным интерфейсом, лояльностью или набор мероприем предоставляемый данным оператором.

---

**Задача 4.1.3.** Опираясь на выводы из задач выше, проверьте продуктовые гипотезы:

- **Гипотеза 1.** Тип мероприятия влияет на вероятность возврата на Яндекс Афишу: пользователи, которые совершили первый заказ на спортивные мероприятия, совершают повторный заказ чаще, чем пользователи, оформившие свой первый заказ на концерты.
- **Гипотеза 2.** В регионах, где больше всего пользователей посещают мероприятия, выше доля повторных заказов, чем в менее активных регионах.

---

Проверка Гипотезы 1: Тип мероприятия

In [ ]:
#Рассчитаем Retention Rate для 'спорт'
# Retention Rate = Среднее значение (доля) столбца 'is_two' для нужной группы
retention_sport = user_profile[user_profile['first_genre'] == 'спорт']['is_two'].mean() * 100

#Рассчитаем Retention Rate для 'концерты'
retention_concerts = user_profile[user_profile['first_genre'] == 'концерты']['is_two'].mean() * 100

print("--- Проверка Гипотезы 1 ---")
print(f"Retention Rate (спорт): {retention_sport:.2f}%")
print(f"Retention Rate (концерты): {retention_concerts:.2f}%")

if retention_sport > retention_concerts:
    print("Гипотеза 1 подтверждается: Спорт > Концерты.")
else:
    print("Гипотеза 1 опровергается: Концерты > Спорт.")
print(f"Разница: {retention_sport - retention_concerts:.2f} п.п.")

Гипотеза 1: О влиянии типа мероприятия Формулировка: Пользователи, чей первый заказ был на спорт, возвращаются чаще, чем те, кто начал с концертов. Проверка по данным: Спорт: Retention Rate = 55.31% Концерты: Retention Rate = 62.11% Вывод: Гипотеза опровергнута. Пользователи, пришедшие через категорию «спорт», показывают самый низкий уровень возврата среди всех категорий. Пользователи «концертов» возвращаются значительно чаще. Вероятно, спортивные болельщики чаще совершают разовые покупки на конкретные матчи,либо влияет сезоность.

Проверка Гипотезы 2: Связь активности и возврата

In [ ]:
#  Группируем по регионам и считаем число пользователей и Retention Rate для каждого региона
region_stats = user_profile.groupby('first_region').agg(
    total_users=('user_id', 'count'),          # Общее число пользователей в регионе
    retention_rate=('is_two', 'mean')          # Среднее is_two (Retention Rate)
).reset_index()
region_stats['retention_rate'] = region_stats['retention_rate'] * 100

# Определяем Топ-3 самых активных региона по total_users
df_sorted = region_stats.sort_values(by='total_users', ascending=False)
top_n = 3 # Устанавливаем N=3, как обсуждалось ранее

top_active_regions = df_sorted.head(top_n)
other_regions = df_sorted.iloc[top_n:]

#  Рассчитываем средний Retention Rate для каждой группы
avg_retention_top = top_active_regions['retention_rate'].mean()
avg_retention_other = other_regions['retention_rate'].mean()

print("\n--- Проверка Гипотезы 2 ---")
print(f"Средний Retention Rate Топ-{top_n} активных регионов: {avg_retention_top:.2f}%")
print(f"Средний Retention Rate Остальных регионов: {avg_retention_other:.2f}%")

if avg_retention_top > avg_retention_other:
    print("Гипотеза 2 подтверждается: Активные регионы > Остальные регионы.")
else:
    print("Гипотеза 2 опровергается: Разницы нет или менее активные регионы > Активные.")
print(f"Разница: {avg_retention_top - avg_retention_other:.2f} п.п.")


Гипотеза 2: В регионах с самым большим количеством пользователей доля повторных заказов выше, чем в менее активных регионах. Проверка по данным (Топ-3 региона по количеству пользователей): Каменевский регион: 7043 пользователь — Retention = 62,28% (чуть выше среднего). Североярская область: 3819 пользователей — Retention =64,38% (высокий). Широковская область: 1247 пользователя — Retention = 64,88% (высокий). Сравнение с менее активными регионами (из списка): Светополянский округ: всего 467 пользователь, но самый высокий Retention = 66,16%. Гипотеза подтверждена частично, но прямой строгой зависимости нет. Хотя крупнейшие регионы (Каменевский и Североярская) показывают результаты выше среднего, самый высокий процент удержания наблюдается в относительно небольшом Светополянском округе (471 чел). При этом регионы с наименьшим числом пользователей в таблице ( Озернинский край) действительно имеют самый низкий Retention.

Общий итог по гипотезе 2: Масштаб региона помогает поддерживать Retention выше среднего, но «лидером лояльности» может быть и небольшой регион с хорошей выборкой по мероприятий, либо ограниченным выбором.

---

#### 4.2. Исследование поведения пользователей через показатели выручки и состава заказа

Изучите количественные характеристики заказов пользователей, чтобы узнать среднюю выручку сервиса с заказа и количество билетов, которое пользователи обычно покупают.

Эти метрики важны не только для оценки выручки, но и для оценки вовлечённости пользователей. Возможно, пользователи с более крупными и дорогими заказами более заинтересованы в сервисе и поэтому чаще возвращаются.

---

**Задача 4.2.1.** Проследите связь между средней выручкой сервиса с заказа и повторными заказами.

- Постройте сравнительные гистограммы распределения средней выручки с билета (`avg_revenue_rub`):
    - для пользователей, совершивших один заказ;
    - для вернувшихся пользователей, совершивших 2 и более заказа.
- Ответьте на вопросы:
    - В каких диапазонах средней выручки концентрируются пользователи из каждой группы?
    - Есть ли различия между группами?

Текст на сером фоне:
    
**Рекомендация:**

1. Используйте одинаковые интервалы (`bins`) и прозрачность (`alpha`), чтобы визуально сопоставить распределения.
2. Задайте параметру `density` значение `True`, чтобы сравнивать форму распределений, даже если число пользователей в группах отличается.

---


Разделим данные на группы

In [ ]:
#  Пользователи с одним заказом
group_one_order = user_profile_filtered[user_profile_filtered['is_two'] == 0]['avg_revenue']

In [ ]:
# Вернувшиеся пользователи (2+ заказов)
group_two_plus_orders = user_profile_filtered[user_profile_filtered['is_two'] == 1]['avg_revenue']

In [ ]:
# Построим  гистограммы
plt.figure(figsize=(12, 6))

plt.hist(
    group_one_order,
    bins=50,
    alpha=0.6,
    density=True,
    label='1 заказ (разовые)',
    color='red'
)
plt.hist(
    group_two_plus_orders,
    bins=50,
    alpha=0.6,
    density=True,
    label='2+ заказа (вернувшиеся)',
    color='blue'
)
#Используем 'avg_revenue' для расчета средних значений и медианы
mean_1 = group_one_order.mean()
median_1 = group_one_order.median()

mean_2 = group_two_plus_orders.mean()
median_2 = group_two_plus_orders.median()

#Добавим на график среднию линию и медиану
plt.axvline(mean_1, color='black', linestyle=':', linewidth=2, label=f'Среднее (1 заказ): {mean_1:.2f}')
plt.axvline(mean_2, color='green', linestyle='--', linewidth=2, label=f'Среднее (2+ заказа): {mean_2:.2f}')
plt.axvline(median_1, color='yellow', linestyle='-', linewidth=2, label=f'Медиана (1): {median_1:.2f}')
plt.axvline(median_2, color='cyan', linestyle='-', linewidth=2, label=f'Медиана (2+): {median_2:.2f}')

plt.title('Сравнение распределения средней выручки с заказа (avg_revenue)')
plt.xlabel('Средняя выручка с заказа, руб.')
plt.ylabel('Плотность распределения (Density)')
plt.legend()
plt.grid(axis='y', alpha=0.5)
plt.show()

# Выводим количественные характеристики для ответа на вопросы
print(f"\nСредняя выручка с заказа (1 заказ): {mean_1:.2f} руб.")
print(f"Средняя выручка с заказа (2+ заказа): {mean_2:.2f} руб.")
print(f"Разница в средней выручке: {mean_2 - mean_1:.2f} руб.")

**Промежуточный вывод**:

Общее сравнение групп Несмотря на то, что средняя выручка в обеих группах практически идентична (около 550 руб.), характер их распределения сильно различается. Группа вернувшихся пользователей (2+ заказа) выглядит более стабильной и предсказуемой.
Ключевые различия в диапазонах Вернувшиеся пользователи (2+) чаще всего совершают покупки в среднем диапазоне от 400 до 800 руб. Разовые пользователи (1 заказ) имеют аномально высокую концентрацию в зоне от 0 до 200 руб., что сильно отличает их от лояльных клиентов.
Смещение (Среднее vs Медиана) В обеих группах наблюдается положительное смещение (среднее значение больше медианы). Это означает, что в данных есть редкие дорогостоящие заказы («хвост» распределения), которые завышают средний чек. У разовых клиентов разрыв между средним (549.23) и медианой (384.72) огромный, что подтверждает сильное влияние выбросов. У вернувшихся клиентов среднее (550.21) и медиана (503.01) намного ближе друг к другу, что говорит о более однородных данных.
Возможные причины Нулевая/низкая выручка у разовых пользователей, скорее всего, связана с возвратами билетов или техническими ошибками при первой покупке. Крупные заказы (высокие чеки) в группе разовых клиентов могут объясняться единоразовыми групповыми покупками (например, покупка билетов на всю компанию или семью один раз).

---

**Задача 4.2.2.** Сравните распределение по средней выручке с заказа в двух группах пользователей:

- совершившие 2–4 заказа;
- совершившие 5 и более заказов.

Ответьте на вопрос: есть ли различия по значению средней выручки с заказа между пользователями этих двух групп?

---


Разделим лояльных пользователей на две группы:

In [ ]:
# Группа 2-4 заказа: те, кто сделал 2+ заказа (is_two == 1), но меньше 5 (is_five == 0)
group_2_4 = user_profile_filtered[(user_profile_filtered['is_two'] == 1) & 
                                  (user_profile_filtered['is_five'] == 0)]['avg_revenue']

In [ ]:
# Группа 5+ заказов: те, у кого установлен признак is_five == 1
group_5_plus = user_profile_filtered[user_profile_filtered['is_five'] == 1]['avg_revenue']

In [ ]:
#Построеним гистограммы
plt.figure(figsize=(12, 6))

plt.hist(
    group_2_4, 
    bins=50, 
    alpha=0.6, 
    density=True, 
    label='2–4 заказа', 
    color='green'
)

plt.hist(
    group_5_plus, 
    bins=50, 
    alpha=0.6, 
    density=True, 
    label='5 и более заказов', 
    color='red'
)

#Расчетаем среднию линию
mean_2_4 = group_2_4.mean()
mean_5_plus = group_5_plus.mean()
 
#Добавим на график среднию линию
plt.axvline(mean_2_4, color='black', linestyle='--', linewidth=2, label=f'Среднее (2-4): {mean_2_4:.2f}')
plt.axvline(mean_5_plus, color='yellow', linestyle='--', linewidth=2, label=f'Среднее (5+): {mean_5_plus:.2f}')


plt.title('Распределение средней выручки: группа 2-4 заказа vs 5+ заказов')
plt.xlabel('Средняя выручка с заказа, руб.')
plt.ylabel('Плотность распределения (Density)')
plt.legend()
plt.grid(axis='y', alpha=0.3)
plt.show()

# Выводим количественные характеристики для ответа на вопросы
print(f"Средняя выручка группы 2-4 заказа: {mean_2_4:.2f} руб.")
print(f"Средняя выручка группы 5+ заказов: {mean_5_plus:.2f} руб.")
print(f"Разница: {mean_5_plus - mean_2_4:.2f} руб.")

**Промежуточный вывод**: Между двумя группами имеются различие. Разница в средних значениях: Средняя выручка у пользователей с 2–4 заказами составляет 555.33 рубля. У самых активных пользователей (5 и более заказов) этот показатель ниже — 544.22 рубля. Таким образом, группа с меньшим количеством заказов в среднем тратит на 11.11 рубля больше за один раз. Где концентрируются пользователи: На графике видно, что основная масса заказов в обеих группах сосредоточена в диапазоне от 500 до 600 рублей. Именно здесь находятся самые высокие столбцы распределения (пики). Форма распределения: Хотя средний чек у группы 5+ чуть ниже, на графике заметно, что их «хвост» распределения (заказы дороже 1000 рублей) выглядит чуть плотнее. Это значит, что они иногда совершают очень крупные покупки, но общую статистику вниз тянет большое количество мелких и недорогих заказов. Итоговый вывод: Высокая лояльность (количество заказов) не означает, что пользователь начинает тратить больше денег за один раз. Самые преданные клиенты (5+ заказов) покупают часто, но выбирают чуть более дешевые варианты, чем те, кто сделал всего 2–4 заказа. Группа 2–4 заказа является самой «дорогой» по среднему чеку среди всех вернувшихся пользователей.

---

**Задача 4.2.3.** Проанализируйте влияние среднего количества билетов в заказе на вероятность повторной покупки.

- Изучите распределение пользователей по среднему количеству билетов в заказе (`avg_tickets_count`) и опишите основные наблюдения.
- Разделите пользователей на несколько сегментов по среднему количеству билетов в заказе:
    - от 1 до 2 билетов;
    - от 2 до 3 билетов;
    - от 3 до 5 билетов;
    - от 5 и более билетов.
- Для каждого сегмента подсчитайте общее число пользователей и долю пользователей, совершивших повторные заказы.
- Ответьте на вопросы:
    - Как распределены пользователи по сегментам — равномерно или сконцентрировано?
    - Есть ли сегменты с аномально высокой или низкой долей повторных покупок?

---

In [ ]:
user_profile_filtered['avg_tickets'].describe().round(2)

In [ ]:
user_profile_filtered = user_profile_filtered.copy()

In [ ]:
def get_ticket_segment(avg_tickets):
    if 1 <= avg_tickets < 2:
        return '1. От 1 до 2 билетов'
    elif 2 <= avg_tickets < 3:
        return '2. От 2 до 3 билетов'
    elif 3 <= avg_tickets < 5:
        return '3. От 3 до 5 билетов'
    else:
        return '4. 5 и более билетов'
    
# Используем .loc для безопасного создания новой колонки
# Символ ':' означает "применить ко всем строкам"
user_profile_filtered.loc[:, 'ticket_segment'] = user_profile_filtered['avg_tickets'].apply(get_ticket_segment)

#  Группируем данные для анализа
ticket_analysis = user_profile_filtered.groupby('ticket_segment').agg(
    total_users=('user_id', 'count'),
    repeat_buyers=('is_two', 'sum'),
    retention_rate=('is_two', 'mean')
).reset_index()

# Рассчитываем долю пользователей в каждом сегменте
total_all_users = ticket_analysis['total_users'].sum()
ticket_analysis['users_share_pct'] = (ticket_analysis['total_users'] / total_all_users * 100).round(2)


print("Анализ влияния среднего количества билетов на повторные покупки:")
display(ticket_analysis.style.format({
    'retention_rate': '{:.2%}',
    'users_share_pct': '{:.2f}%'
}))

In [ ]:
# Строим график
plt.figure(figsize=(12, 6))
sns.barplot(data=ticket_analysis, x='ticket_segment', y='retention_rate', palette='viridis')
plt.axhline(user_profile_filtered['is_two'].mean(), color='red', linestyle='--', label='Среднее по всей базе')
plt.title('Доля повторных покупок в зависимости от среднего кол-ва билетов')
plt.ylabel('Retention Rate (Доля 2+ заказов)')
plt.xlabel('Сегмент по среднему кол-ву билетов в заказе')
plt.legend()
plt.show()

**Промежуточный вывод**:

Распределение пользователей сконцентрировано в двух центральных сегментах: "От 2 до 3 билетов" (44.20%) и "От 3 до 5 билетов" (42.40%). Вместе эти два сегмента составляют более 86% всей пользовательской базы. Сегмент "1 до 2 билетов" и сегмент "5 и более билетов" являются наименее многочисленными (11.23% и 2.17% соответственно).

В процессе анализа выявлены два сегмента с ярко выраженными аномалиями: Аномально высокая доля повторных покупок: Сегмент "От 2 до 3 билетов" имеет долю повторных покупок 73.46%, что значительно выше среднего по базе (около 61–62%). Эти пользователи являются наиболее лояльными и возвратными. Аномально низкая доля повторных покупок: Сегмент "5 и более билетов" имеет крайне низкую долю повторных покупок — всего 13.17%. Это свидетельствует о том, что крупные групповые покупки, как правило, являются разовыми событиями.

Среднее количество билетов в заказе является основным показателем возрата за повторной покупкой билета.

---

#### 4.3. Исследование временных характеристик первого заказа и их влияния на повторные покупки

Изучите временные параметры, связанные с первым заказом пользователей:

- день недели первой покупки;
- время с момента первой покупки — лайфтайм;
- средний интервал между покупками пользователей с повторными заказами.

---

**Задача 4.3.1.** Проанализируйте, как день недели, в которой была совершена первая покупка, влияет на поведение пользователей.

- По данным даты первого заказа выделите день недели.
- Для каждого дня недели подсчитайте общее число пользователей и долю пользователей, совершивших повторные заказы. Результаты визуализируйте.
- Ответьте на вопрос: влияет ли день недели, в которую совершена первая покупка, на вероятность возврата клиента?

---


In [ ]:
# Словарь для сопоставления числового дня недели (0=Пн, 6=Вс) с русскими названиями
days_mapping = {0: 'Понедельник', 1: 'Вторник', 2: 'Среда', 3: 'Четверг', 4: 'Пятница', 5: 'Суббота', 6: 'Воскресенье'}

# Создаем колонку 'day_of_week', используя .loc для избежания SettingWithCopyWarning
user_profile_filtered.loc[:, 'day_of_week'] = user_profile_filtered['first_order_date'].dt.dayofweek.map(days_mapping)

#Группировка и агрегация данных по дню недели
day_of_week_analysis = user_profile_filtered.groupby('day_of_week').agg(
    total_users=('user_id', 'count'),
    repeat_buyers=('is_two', 'sum'),
    retention_rate=('is_two', 'mean')
).reset_index()

#  Упорядочивание дней недели для правильной визуализации
day_order = ['Понедельник', 'Вторник', 'Среда', 'Четверг', 'Пятница', 'Суббота', 'Воскресенье']
day_of_week_analysis['day_of_week'] = pd.Categorical(
    day_of_week_analysis['day_of_week'], 
    categories=day_order, 
    ordered=True
)

In [ ]:
# Сортируем по дням недели
day_of_week_analysis = day_of_week_analysis.sort_values('day_of_week')

In [ ]:
print("Влияние дня недели первой покупки на Retention Rate:")
display(day_of_week_analysis.style.format({
    'retention_rate': '{:.2%}',
    'total_users': '{:,}',
    'repeat_buyers': '{:,}'
}))

In [ ]:
# Расчет среднего уровня Retention Rate по всей базе для горизонтальной линии
overall_retention = user_profile_filtered['is_two'].mean()

In [ ]:
plt.figure(figsize=(12, 6))

# Для визуализации используем упорядоченный столбец

sns.barplot(data=day_of_week_analysis, x='day_of_week', y='retention_rate', palette='Pastel1')
plt.axhline(overall_retention, color='red', linestyle='--', label=f'Среднее по всей базе ({overall_retention:.2%})')
plt.title('Доля повторных покупок в зависимости от дня недели первой покупки')
plt.ylabel('Retention Rate (Доля 2+ заказов)')
plt.xlabel('День недели первой покупки')
plt.legend()
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', alpha=0.5)
plt.tight_layout()
plt.show()

**Промежуточный вывод**: День недели практически не влияет на долю повторных покупок

---

**Задача 4.3.2.** Изучите, как средний интервал между заказами влияет на удержание клиентов.

- Рассчитайте среднее время между заказами для двух групп пользователей:
    - совершившие 2–4 заказа;
    - совершившие 5 и более заказов.
- Исследуйте, как средний интервал между заказами влияет на вероятность повторного заказа, и сделайте выводы.

---


In [ ]:
 # Фильтруем пользователей с повторными заказами, у которых рассчитан avg_days_between
# (avg_days_between рассчитан только для total_orders > 1)
loyal_users = user_profile_filtered[user_profile_filtered['is_two'] == 1].copy()

# Убедимся, что колонка с интервалом не содержит NaN (для пользователей с одним заказом)
loyal_users = loyal_users.dropna(subset=['avg_days_between'])

In [ ]:
# Разделение лояльных пользователей на две группы по общему количеству заказов
# Группа 2-4 заказа: is_two == 1 И is_five == 0
group_2_4_orders = loyal_users[loyal_users['is_five'] == 0]

# Группа 5+ заказов: is_five == 1
group_5_plus_orders = loyal_users[loyal_users['is_five'] == 1]

In [ ]:
#Расчетаем средний интервал между покупками для каждой группы
mean_days_2_4 = group_2_4_orders['avg_days_between'].mean()
mean_days_5_plus = group_5_plus_orders['avg_days_between'].mean()

print(f"Средний интервал между заказами для группы 2-4 заказа: {mean_days_2_4:.2f} дней")
print(f"Средний интервал между заказами для группы 5+ заказов: {mean_days_5_plus:.2f} дней")

In [ ]:
# Строим график
plt.figure(figsize=(12,8))
plt.hist(
    group_2_4_orders['avg_days_between'],
    bins=30,
    alpha=0.6,
    density=True,
    label=f'Группа 2-4 заказа (Среднее: {mean_days_2_4:.2f} дн.)',
    color='darkorange'
)

# Гистограмма для группы 5+ заказов
plt.hist(
    group_5_plus_orders['avg_days_between'],
    bins=20,
    alpha=0.6,
    density=True,
    label=f'Группа 5+ заказов (Среднее: {mean_days_5_plus:.2f} дн.)',
    color='darkblue'
)
plt.axvline(mean_days_2_4, color='darkorange', linestyle='--', linewidth=2) #добавим средний интервал
plt.axvline(mean_days_5_plus, color='darkblue', linestyle='--', linewidth=2)

plt.title('Сравнение распределения среднего интервала между заказами')
plt.xlabel('Средний интервал между покупками (дни)')
plt.ylabel('Плотность распределения (Density)')
plt.legend()
plt.grid(axis='y', alpha=0.5)
plt.show()

**Промежуточный вывод**: Пользователи с 2-4 заказами в среднем возвращаются через 21.33 дней. Пользователи с 5+ заказами в среднем возвращаются через 9.91 дней, разница между группами: 11.42 дней. Чем чаще пользователь возвращается (меньше интервал), тем выше общее количество заказов. ПОЛЬЗОВАТЕЛИ С БОЛЕЕ ЧАСТЫМИ ПОКУПКАМИ (меньший интервал):

Совершают значительно больше заказов в целом
Демонстрируют более высокую вовлеченность
Являются наиболее ценной группой клиентов ПОЛЬЗОВАТЕЛИ С РЕДКИМИ ПОКУПКАМИ (большой интервал):
Рискуют "отвалиться" после 2-4 заказов
Требуют дополнительных усилий для удержания
Интервал между заказами - важный индикатор лояльности. Пользователи, возвращающиеся быстрее, становятся наиболее ценными клиентами

---

#### 4.4. Корреляционный анализ количества покупок и признаков пользователя

Изучите, какие характеристики первого заказа и профиля пользователя могут быть связаны с числом покупок. Для этого используйте универсальный коэффициент корреляции `phi_k`, который позволяет анализировать как числовые, так и категориальные признаки.

---

**Задача 4.4.1:** Проведите корреляционный анализ:
- Рассчитайте коэффициент корреляции `phi_k` между признаками профиля пользователя и числом заказов (`total_orders`). При необходимости используйте параметр `interval_cols` для определения интервальных данных.
- Проанализируйте полученные результаты. Если полученные значения будут близки к нулю, проверьте разброс данных в `total_orders`. Такое возможно, когда в данных преобладает одно значение: в таком случае корреляционный анализ может показать отсутствие связей. Чтобы этого избежать, выделите сегменты пользователей по полю `total_orders`, а затем повторите корреляционный анализ. Выделите такие сегменты:
    - 1 заказ;
    - от 2 до 4 заказов;
    - от 5 и выше.
- Визуализируйте результат корреляции с помощью тепловой карты.
- Ответьте на вопрос: какие признаки наиболее связаны с количеством заказов?

---

In [ ]:
# Используем отфильтрованный датафрейм user_profile_filtered
df = user_profile_filtered.copy()

In [ ]:
features_to_correlate =['first_device',     
    'first_region',     
    'first_partner',    
    'first_genre',      
    'avg_revenue', 
    'avg_tickets', 
    'avg_days_between', 
    'total_orders'
]
# Создадим подмножество данных
df_subset = df[features_to_correlate]

# Определим интервальные (числовые) колонки для phik
interval_cols = ['avg_revenue', 'avg_tickets', 'avg_days_between', 'total_orders']

print("--- Корреляционный анализ с исходным признаком 'total_orders' ---")

In [ ]:
# Расчет матрицы phi_k
phik_corr_original = df_subset.phik_matrix(interval_cols=interval_cols)

In [ ]:
# Визуализация (только для целевого признака)
correlation_original = phik_corr_original.loc['total_orders'].sort_values(ascending=False).to_frame()

display(correlation_original.style.format('{:.3f}'))

In [ ]:
# Проверим разброс total_orders
print(f"\nРазброс (минимум/максимум) total_orders: {df['total_orders'].min()} / {df['total_orders'].max()}")
print(f"Мода total_orders (наиболее частое значение): {df['total_orders'].mode().iloc[0]}")

# Создаем категориальный признак 'orders_segment'
def segment_orders(total_orders):
    if total_orders == 1:
        return '1. 1 заказ'
    elif 2 <= total_orders <= 4:
        return '2. От 2 до 4 заказов'
    elif total_orders >= 5:
        return '3. 5 и более заказов'
    else:
        return np.nan

# Создаем 'orders_segment' с использованием .loc для предотвращения SettingWithCopyWarning
df.loc[:, 'orders_segment'] = df['total_orders'].apply(segment_orders)

In [ ]:
# СПИСОК ПРИЗНАКОВ для сегментированного анализа
segmented = [
    'first_device',
    'first_region', 
    'first_partner',
    'first_genre', 
    'avg_revenue',
    'avg_tickets', 
    'avg_days_between',
    'orders_segment'
]

df_segmented_subset = df[segmented]

# Расчет матрицы phi_k для сегментированных данных (цель: 'orders_segment')
phik_corr_segmented = df_segmented_subset.phik_matrix(interval_cols=['avg_revenue', 'avg_tickets', 'avg_days_between'])

In [ ]:
#  Визуализация корреляции с целевым признаком 'orders_segment'
correlation = phik_corr_segmented.loc['orders_segment'].drop('orders_segment').sort_values(ascending=False).to_frame()

print("\n--- Корреляционный анализ с сегментированным признаком 'orders_segment' ---")
display(correlation.style.format('{:.3f}'))

#  Визуализация полной матрицы с использованием seaborn
plt.figure(figsize=(12, 10))
sns.heatmap(phik_corr_segmented.fillna(0), 
            annot=True, 
            fmt=".2f", 
            cmap='RdBu_r', 
            center=0)

plt.title('Корреляционная матрица Phi-K', fontsize=15)
plt.show()

**Промежуточный вывод**: Анализ факторов, влияющих на количество заказов Проведение сегментации целевого признака total_orders (разделение на «1 заказ», «2-4 заказа» и «5 и более») позволило значительно лучше выявить скрытые зависимости, которые были менее заметны в исходных данных. Коэффициент хорошо улавливает как линейные, так и нелинейные связи.

Ключевые факторы корреляции . Среднее количество билетов (avg_tickets). Наблюдается четкая зависимость: чем больше билетов пользователь покупает за один раз, тем выше вероятность, что он попадет в сегмент лояльных клиентов с большим количеством заказов. Это может указывать на то, что «семейные» клиенты или те, кто покупает билеты на компании, возвращаются чаще. Среднее время между заказами (avg_days_between) . Данный показатель ожидаемо влияет на сегментацию лояльности. Пользователи с меньшим интервалом между покупками быстрее переходят в категорию многократно заказывающих.
Второстепенные факторы. Средняя выручка (avg_revenue). Хотя объем прибыли важен, он влияет на частоту заказов меньше, чем количество билетов в чеке. Региональная привязка (first_region). В некоторых регионах инфраструктура или маркетинговая активность способствуют удержанию клиентов лучше, чем в других.
Незначительные факторы Жанр и партнер ( first_genre, first_partner): Эти признаки имеют практически нулевую или крайне низкую корреляцию ( ) с сегментом заказов. Это означает, что то, с какого устройства зашел пользователь или какой фильм/спектакль он выбрал в первый раз, почти не влияет на то, станет ли он постоянным клиентом. Итог для моделирования: Для предсказания лояльности (количества будущих заказов) наиболее ценными признаками являются avg_tickets и avg_days_between. Признаки, связанные с первым опытом (партнером, жанр), можно считать малоинформативными для данной задачи и, возможно, исключить из модели для её упрощения.

### 5. Общий вывод и рекомендации

В конце проекта напишите общий вывод и рекомендации: расскажите заказчику, на что нужно обратить внимание. В выводах кратко укажите:

- **Информацию о данных**, с которыми вы работали, и то, как они были подготовлены: например, расскажите о фильтрации данных, переводе тенге в рубли, фильтрации выбросов.
- **Основные результаты анализа.** Например, укажите:
    - Сколько пользователей в выборке? Как распределены пользователи по числу заказов? Какие ещё статистические показатели вы подсчитали важным во время изучения данных?
    - Какие признаки первого заказа связаны с возвратом пользователей?
    - Как связаны средняя выручка и количество билетов в заказе с вероятностью повторных покупок?
    - Какие временные характеристики влияют на удержание (день недели, интервалы между покупками)?
    - Какие характеристики первого заказа и профиля пользователя могут быть связаны с числом покупок согласно результатам корреляционного анализа?
- Дополните выводы информацией, которая покажется вам важной и интересной. Следите за общим объёмом выводов — они должны быть компактными и ёмкими.

В конце предложите заказчику рекомендации о том, как именно действовать в его ситуации. Например, укажите, на какие сегменты пользователей стоит обратить внимание в первую очередь, а какие нуждаются в дополнительных маркетинговых усилиях.

**Исходные данные**:

Анализ проводился на основе истории заказов билетов (исключая сегмент «фильмы») с общим объемом около 290 тысяч записей, которые были агрегированы в 21 700 уникальных профилей пользователей. Подготовка:
Данные были тщательно очищены и стандартизированы. Произведены следующие ключевые шаги:
Конвертация валют: Вся выручка приведена к единой валюте — российскому рублю (revenue_rub).
Очистка аномалий и выбросов: Удалены строки с отрицательной/нулевой выручкой, а также экстремально высокие выбросы (заказы дороже 99-го перцентиля и более 5 билетов в среднем).
Генерация признаков: Рассчитаны ключевые метрики, включая количество дней с предыдущего заказа (days_since_prior_order), что позволило агрегировать всю историю клиента в одну строку.
Общий процент удаленных данных составил всего 3.06%, что гарантирует высокую достоверность статистики.

**Основные результаты анализа** Метрика Значение Интерпретация Пользователей в выборке 21,700 Достаточный объем для надежных выводов. Уровень лояльности (2+ заказа) 61.72% Очень высокий уровень удержания. Среднее число заказов 6.45 Средняя частота покупок для лояльных клиентов. Средний чек ~550 RUB Лояльность не связана с высоким разовым чеком, а с частотой покупок в умеренном ценовом диапазоне (500-750 RUB).

Жанр первого мероприятия (Retention Rate): Лидеры: Концерты (самый большой объем) и Выставки/Театр (самый высокий возврат). Аутсайдеры: Спорт и «Ёлки» (низкий возврат). Регион и Партнер: Имеют заметное влияние на возврат, с явными регионами-лидерами (Светополянский округ) и партнерами-лидерами («Дом культуры»).

Средняя выручка (avg_revenue): Связь непрямая. Самая «дорогая» группа по среднему чеку — клиенты, совершившие всего 2–4 заказа (550.36 RUB). Самые лояльные клиенты (5+ заказов) тратят меньше (544.65 RUB).

Вывод: высокая лояльность обусловлена не стоимостью покупки, а частотой покупок недорогих и средних билетов. Количество билетов (avg_tickets): Критически важный фактор.

Пользователи, покупающие 2–3 билета за раз, имеют самый высокий возврат Низкий возврат имеют клиенты, покупающие 5 и более билетов за раз, редко возвращаются, что указывает на то, что это разовые корпоративные или экскурсионные заказы. День недели первой покупки не оказывает значительного влияния. Разница в Retention Rate между лучшим и худшим днем составляет всего 3.4% процентных пункта (59.7%–63.05%). Средний интервал между покупками (avg_days_between): Критически важен. Пользователи с 5+ заказами возвращаются в среднем каждые 9.91 дней. Пользователи с 2–4 заказами возвращаются реже — каждые 21.33 дней. Наиболее сильными предикторами количества заказов являются:

Среднее количество билетов в заказе (avg_tickets): (очень сильная связь). Средний интервал между покупками (avg_days_between): (умеренно сильная связь). Признаки, связанные с первым опытом (устройство, жанр, партнер), имеют очень низкую корреляцию с итоговым количеством заказов (лояльностью).

**Общие рекомендации заказчику** Для повышения лояльности и выручки необходимо сфокусироваться на следующих стратегических направлениях:

1.Фокус на лояльность (Сегмент 2–3 билета) Продукт и маркетинг: Признать, что пара «2–3 билета» (пары, небольшие семьи) является основой лояльной базы. Рекламные кампании, программы лояльности и специальные предложения должны быть ориентированы именно на эту группу. Удержание: Стимулировать этих клиентов к повторным покупкам, используя их средний интервал между покупками как ключевой показатель.

2.Сокращение интервала покупок Целевая группа: Группа «2–4 заказа» имеет длительный средний интервал и является самой «дорогой» по чеку. Тактика: Запускать персонализированные предложения (специальные подборки, скидки на второй билет), которые срабатывают до наступления среднего интервала в 12 дней. Цель — приблизить частоту покупок этой группы к показателям группы «5+ заказов».

3.Пересмотр работы с крупными заказами Группа «5+ билетов»: Эти клиенты крайне нелояльны. Тактика: Считать их разовыми клиентами. Перенести фокус с удержания на увеличение среднего чека в рамках этой единственной покупки. Для этой группы эффективны могут быть B2B-предложения или предложения, включающие дополнительные услуги (VIP-доступ, лучшие места).

### 6. Финализация проекта и публикация в Git

Когда вы закончите анализировать данные, оформите проект, а затем опубликуйте его.

Выполните следующие действия:

1. Создайте файл `.gitignore`. Добавьте в него все временные и чувствительные файлы, которые не должны попасть в репозиторий.
2. Сформируйте файл `requirements.txt`. Зафиксируйте все библиотеки, которые вы использовали в проекте.
3. Вынести все чувствительные данные (параметры подключения к базе) в `.env`файл.
4. Проверьте, что проект запускается и воспроизводим.
5. Загрузите проект в публичный репозиторий — например, на GitHub. Убедитесь, что все нужные файлы находятся в репозитории, исключая те, что в `.gitignore`. Ссылка на репозиторий понадобится для отправки проекта на проверку. Вставьте её в шаблон проекта в тетрадке Jupyter Notebook перед отправкой проекта на ревью.

**Вставьте ссылку на проект в этой ячейке тетрадки перед отправкой проекта на ревью.**
